### Import section 

In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import plotly.express as px
import pickle
import tensorflow as tf
import os

sns.set()
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator


from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer, LabelEncoder, StandardScaler, MinMaxScaler, MaxAbsScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn import set_config
from sklearn.base import TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit



from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


In [125]:
storeFirstDataframe = pd.read_csv('./rawData/data_prueba_Forecasting.csv')

In [126]:
storeFirstDataframe.head(100)

,Unnamed: 0,Formato,Subformato,Tienda,Producto,id_fec_diaria,Venta,Precio,Venta_aa,Precio_aa
0,0,Super,Basico,1,369772,2021-08-01,7.000000,17.168571,5.000000,21.063200
1,1,Super,Basico,1,369772,2021-08-02,6.000000,17.141667,3.000000,21.388667
2,2,Super,Basico,1,369772,2021-08-03,5.000000,17.136000,5.000000,21.376199
3,3,Super,Basico,1,369772,2021-08-04,4.000000,16.995000,3.000000,21.360367
4,4,Super,Basico,1,369772,2021-08-05,1.000000,17.240000,1.000000,21.397100
...,...,...,...,...,...,...,...,...,...,...
95,18,Hiper,Plus,1,303923,2021-08-19,54.124001,112.441242,36.127999,94.703066
96,19,Hiper,Plus,1,303923,2021-08-20,49.636001,112.436133,40.871000,95.028258
97,20,Hiper,Plus,1,303923,2021-08-21,82.614000,112.669526,64.603002,94.451203
98,21,Hiper,Plus,1,303923,2021-08-22,97.948002,112.506122,85.565998,94.756450


In [127]:
storeFirstDataframe.tail()

,Unnamed: 0,Formato,Subformato,Tienda,Producto,id_fec_diaria,Venta,Precio,Venta_aa,Precio_aa
426372,87,Super,Plus,1,2098857,2021-10-27,8.0,27.003750,16.0,20.083056
426373,88,Super,Plus,1,2098857,2021-10-28,18.0,26.911667,22.0,20.121114
426374,89,Super,Plus,1,2098857,2021-10-29,13.0,24.463077,20.0,18.967170
426375,90,Super,Plus,1,2098857,2021-10-30,17.0,25.404118,37.0,18.206573
426376,91,Super,Plus,1,2098857,2021-10-31,23.0,23.815652,24.0,18.436345


In [128]:
storeFirstDataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426377 entries, 0 to 426376
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     426377 non-null  int64  
 1   Formato        426377 non-null  object 
 2   Subformato     426377 non-null  object 
 3   Tienda         426377 non-null  int64  
 4   Producto       426377 non-null  int64  
 5   id_fec_diaria  426377 non-null  object 
 6   Venta          426377 non-null  float64
 7   Precio         426377 non-null  float64
 8   Venta_aa       426377 non-null  float64
 9   Precio_aa      426377 non-null  float64
dtypes: float64(4), int64(3), object(3)
memory usage: 32.5+ MB



Como no tenemos valores nulos, no hay necesidad de hacer ningún tipo de data imputation.

Igualmente tenemos que el tipo de datos es justamente lo que requerimos para cada columna, excepto para las fechas que deberían ser tipo datetime, por ende tendremos qué cambiar las mismas.


In [129]:
storeFirstDataframe['id_fec_diaria'] = pd.to_datetime(storeFirstDataframe['id_fec_diaria']) 
def fill_missing(values):
    one_day = 60*24
    for row in range(storeFirstDataframe.shape[0]):
        for col in range(storeFirstDataframe.shape[1]):
            if pd.isnull(values[row][col]):
                values[row,col] = values[row-one_day,col]
fill_missing(storeFirstDataframe.values)
storeFirstDataframeDateChanged = storeFirstDataframe.copy()
storeFirstDataframeDateChanged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426377 entries, 0 to 426376
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Unnamed: 0     426377 non-null  int64         
 1   Formato        426377 non-null  object        
 2   Subformato     426377 non-null  object        
 3   Tienda         426377 non-null  int64         
 4   Producto       426377 non-null  int64         
 5   id_fec_diaria  426377 non-null  datetime64[ns]
 6   Venta          426377 non-null  float64       
 7   Precio         426377 non-null  float64       
 8   Venta_aa       426377 non-null  float64       
 9   Precio_aa      426377 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(2)
memory usage: 32.5+ MB


In [130]:
storeFirstDataframeDateChanged.isnull().sum()

Unnamed: 0       0
Formato          0
Subformato       0
Tienda           0
Producto         0
id_fec_diaria    0
Venta            0
Precio           0
Venta_aa         0
Precio_aa        0
dtype: int64

In [131]:
storeFirstDataframeDateChanged.describe()

,Unnamed: 0,Tienda,Producto,Venta,Precio,Venta_aa,Precio_aa
count,426377.000000,426377.0,4.263770e+05,426377.000000,426377.000000,426377.000000,426377.000000
mean,45.552659,1.0,5.120868e+06,68.534217,61.782508,73.905263,58.161344
std,26.532954,0.0,4.911189e+06,319.971154,70.248355,348.099302,66.369279
min,0.000000,1.0,1.160000e+02,0.076000,0.870000,0.084000,0.008500
25%,23.000000,1.0,1.035460e+06,4.000000,23.902857,4.000000,22.561006
50%,46.000000,1.0,2.101039e+06,11.000000,40.270741,12.000000,38.382699
75%,69.000000,1.0,1.120037e+07,38.000000,74.882001,41.000000,68.401846
max,91.000000,1.0,1.153318e+07,41883.469971,1650.859985,46097.408234,1591.921753


In [132]:
storeFirstDataframeDateChanged.describe(include='all')

C:\Users\alan_\AppData\Local\Temp\ipykernel_26496\2635487392.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,Unnamed: 0,Formato,Subformato,Tienda,Producto,id_fec_diaria,Venta,Precio,Venta_aa,Precio_aa
count,426377.000000,426377,426377,426377.0,4.263770e+05,426377,426377.000000,426377.000000,426377.000000,426377.000000
unique,NaN,2,3,NaN,NaN,92,NaN,NaN,NaN,NaN
top,NaN,Hiper,Basico,NaN,NaN,2021-08-14 00:00:00,NaN,NaN,NaN,NaN
freq,NaN,323985,170484,NaN,NaN,4761,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,2021-08-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2021-10-31 00:00:00,NaN,NaN,NaN,NaN
mean,45.552659,NaN,NaN,1.0,5.120868e+06,NaN,68.534217,61.782508,73.905263,58.161344
std,26.532954,NaN,NaN,0.0,4.911189e+06,NaN,319.971154,70.248355,348.099302,66.369279
min,0.000000,NaN,NaN,1.0,1.160000e+02,NaN,0.076000,0.870000,0.084000,0.008500
25%,23.000000,NaN,NaN,1.0,1.035460e+06,NaN,4.000000,23.902857,4.000000,22.561006


In [133]:
ValoresSobreLaMediaDeVenta = storeFirstDataframeDateChanged[storeFirstDataframeDateChanged['Precio']>storeFirstDataframeDateChanged.Venta.mean()]['Producto'].unique().size
ValoresSobreLaMediaDeVenta

945

In [134]:
TotalProducts = storeFirstDataframeDateChanged.Producto.unique().shape[0]
TotalProducts

2405

In [135]:
uniqueProducts = pd.Series(storeFirstDataframe.Producto.unique())
uniqueProducts

0         369772
1         303923
2        1791075
3        1720010
4        1224404
          ...   
2400      378759
2401    11442635
2402    11400275
2403    11025261
2404     1155395
Length: 2405, dtype: int64

In [136]:
ValoresSobreLaMediaDeVenta/TotalProducts

0.39293139293139295

Tenemos que son 945 productos los que nos dan valores mayores a la media, que equivalen al 39%.  

Ahora vamos a llevar a cabo el one hot encoding para las categorías de formato y subformato, y también para tiempo.

In [137]:
(storeFirstDataframeDateChanged.duplicated() == 'true').count()


426377

In [138]:
duplicatedRows = storeFirstDataframeDateChanged.groupby(storeFirstDataframeDateChanged.columns.tolist(),as_index=False).size()
duplicatedRows


,Unnamed: 0,Formato,Subformato,Tienda,Producto,id_fec_diaria,Venta,Precio,Venta_aa,Precio_aa,size
0,0,Hiper,Basico,1,116,2021-08-01,2335.494020,68.906064,2583.950001,48.858814,1
1,0,Hiper,Basico,1,348,2021-08-01,2774.085018,27.362979,2609.015009,26.273482,1
2,0,Hiper,Basico,1,407,2021-08-01,3653.395016,32.139943,4579.670032,26.264912,1
3,0,Hiper,Basico,1,456,2021-08-01,6582.344982,18.987914,5421.385014,21.369818,1
4,0,Hiper,Basico,1,480,2021-08-01,5486.099964,33.628015,5593.035027,28.991718,1
...,...,...,...,...,...,...,...,...,...,...,...
426372,91,Super,Plus,1,11498347,2021-10-31,5.000000,27.512000,4.000000,27.424850,1
426373,91,Super,Plus,1,11499565,2021-10-31,9.000000,67.826669,4.000000,57.490750,1
426374,91,Super,Plus,1,11501938,2021-10-31,8.000000,23.821249,5.000000,19.696220,1
426375,91,Super,Plus,1,11515961,2021-10-31,5.000000,66.176001,6.000000,75.181699,1


In [139]:
total_rows = storeFirstDataframeDateChanged.shape[0]

In [140]:
storeSecondDataframeDateChanged = storeFirstDataframeDateChanged.copy()

storeSecondDataframeDateChanged['id_fec_diaria'] = storeSecondDataframeDateChanged['id_fec_diaria'].apply(lambda x: x.replace(year = x.year - 1))

storeSecondDataframeDateChanged=storeSecondDataframeDateChanged.drop(columns=['Venta', 'Precio'])

storeSecondDataframeDateChanged.rename(columns = {'Venta_aa':'Venta', 'Precio_aa':'Precio'}, inplace = True)

storeFirstDataframeDateChanged = storeFirstDataframeDateChanged.drop(columns=['Venta_aa', 'Precio_aa'])

twoYearsDataframe = [storeSecondDataframeDateChanged,storeFirstDataframeDateChanged]

twoYearsDataframeConcat = pd.concat(twoYearsDataframe, ignore_index=True)

twoYearsDataframeConcat


,Unnamed: 0,Formato,Subformato,Tienda,Producto,id_fec_diaria,Venta,Precio
0,0,Super,Basico,1,369772,2020-08-01,5.0,21.063200
1,1,Super,Basico,1,369772,2020-08-02,3.0,21.388667
2,2,Super,Basico,1,369772,2020-08-03,5.0,21.376199
3,3,Super,Basico,1,369772,2020-08-04,3.0,21.360367
4,4,Super,Basico,1,369772,2020-08-05,1.0,21.397100
...,...,...,...,...,...,...,...,...
852749,87,Super,Plus,1,2098857,2021-10-27,8.0,27.003750
852750,88,Super,Plus,1,2098857,2021-10-28,18.0,26.911667
852751,89,Super,Plus,1,2098857,2021-10-29,13.0,24.463077
852752,90,Super,Plus,1,2098857,2021-10-30,17.0,25.404118


Igualmente no tenemos ningún registro duplicado. Ahora lo que haremos es extraer los meses, días y años, habiendo extendido nuestro dataframe con los valores del año pasado.

In [141]:
list_months = []
list_days = []
list_years = []
list_months
list_days
list_years

[]

In [142]:
twoYearsDataframeConcat['id_fec_diaria']

0        2020-08-01
1        2020-08-02
2        2020-08-03
3        2020-08-04
4        2020-08-05
            ...    
852749   2021-10-27
852750   2021-10-28
852751   2021-10-29
852752   2021-10-30
852753   2021-10-31
Name: id_fec_diaria, Length: 852754, dtype: datetime64[ns]

In [143]:
new_total_rows = twoYearsDataframeConcat.shape[0]

In [144]:
for i in range(new_total_rows):
    list_months.append(twoYearsDataframeConcat['id_fec_diaria'][i].month)
    
for j in range(new_total_rows):
    list_days.append(twoYearsDataframeConcat['id_fec_diaria'][j].day)
    
for k in range(new_total_rows):
    list_years.append(twoYearsDataframeConcat['id_fec_diaria'][k].year)

In [145]:
list_months = pd.Series(list_months)
list_days = pd.Series(list_days)
list_years = pd.Series(list_years)

In [146]:
twoYearsDataframeConcat.head()

,Unnamed: 0,Formato,Subformato,Tienda,Producto,id_fec_diaria,Venta,Precio
0,0,Super,Basico,1,369772,2020-08-01,5.0,21.063200
1,1,Super,Basico,1,369772,2020-08-02,3.0,21.388667
2,2,Super,Basico,1,369772,2020-08-03,5.0,21.376199
3,3,Super,Basico,1,369772,2020-08-04,3.0,21.360367
4,4,Super,Basico,1,369772,2020-08-05,1.0,21.397100


In [147]:
twoYearsDataframeConcat.tail()

,Unnamed: 0,Formato,Subformato,Tienda,Producto,id_fec_diaria,Venta,Precio
852749,87,Super,Plus,1,2098857,2021-10-27,8.0,27.003750
852750,88,Super,Plus,1,2098857,2021-10-28,18.0,26.911667
852751,89,Super,Plus,1,2098857,2021-10-29,13.0,24.463077
852752,90,Super,Plus,1,2098857,2021-10-30,17.0,25.404118
852753,91,Super,Plus,1,2098857,2021-10-31,23.0,23.815652


In [148]:
twoYearsDataframeConcat['Mes'] = list_months
twoYearsDataframeConcat['Day'] = list_days
twoYearsDataframeConcat['Year'] = list_years

In [149]:
twoYearsDataframeConcat.head()

,Unnamed: 0,Formato,Subformato,Tienda,Producto,id_fec_diaria,Venta,Precio,Mes,Day,Year
0,0,Super,Basico,1,369772,2020-08-01,5.0,21.063200,8,1,2020
1,1,Super,Basico,1,369772,2020-08-02,3.0,21.388667,8,2,2020
2,2,Super,Basico,1,369772,2020-08-03,5.0,21.376199,8,3,2020
3,3,Super,Basico,1,369772,2020-08-04,3.0,21.360367,8,4,2020
4,4,Super,Basico,1,369772,2020-08-05,1.0,21.397100,8,5,2020


In [150]:
twoYearsDataframeConcat.tail()

,Unnamed: 0,Formato,Subformato,Tienda,Producto,id_fec_diaria,Venta,Precio,Mes,Day,Year
852749,87,Super,Plus,1,2098857,2021-10-27,8.0,27.003750,10,27,2021
852750,88,Super,Plus,1,2098857,2021-10-28,18.0,26.911667,10,28,2021
852751,89,Super,Plus,1,2098857,2021-10-29,13.0,24.463077,10,29,2021
852752,90,Super,Plus,1,2098857,2021-10-30,17.0,25.404118,10,30,2021
852753,91,Super,Plus,1,2098857,2021-10-31,23.0,23.815652,10,31,2021


In [151]:
twoYearsDataframeConcatplot = twoYearsDataframeConcat.copy()

twoYearsDataframeConcatplot['id_fec_diaria'] = twoYearsDataframeConcatplot['id_fec_diaria']

daily_avg_sale = twoYearsDataframeConcatplot.groupby('id_fec_diaria')['Venta'].mean().reset_index()

fig = px.line(daily_avg_sale, x='id_fec_diaria', y='Venta',  title="Daily Average Sale",
              labels={'id_fec_diaria': 'Date', 'Venta': 'Average Sale (Venta)'})

fig.show()


In [152]:
storeFirstDataframeFeatureDates = twoYearsDataframeConcat.copy()

In [153]:
storeFirstDataframeFeatureDates.head()

,Unnamed: 0,Formato,Subformato,Tienda,Producto,id_fec_diaria,Venta,Precio,Mes,Day,Year
0,0,Super,Basico,1,369772,2020-08-01,5.0,21.063200,8,1,2020
1,1,Super,Basico,1,369772,2020-08-02,3.0,21.388667,8,2,2020
2,2,Super,Basico,1,369772,2020-08-03,5.0,21.376199,8,3,2020
3,3,Super,Basico,1,369772,2020-08-04,3.0,21.360367,8,4,2020
4,4,Super,Basico,1,369772,2020-08-05,1.0,21.397100,8,5,2020


In [154]:
storeFirstDataframeFeatureDates = pd.get_dummies(storeFirstDataframeFeatureDates,
                                            columns = ['Formato', 'Subformato'], drop_first=False)
storeFirstDataframeFeatureDates


,Unnamed: 0,Tienda,Producto,id_fec_diaria,Venta,Precio,Mes,Day,Year,Formato_Hiper,Formato_Super,Subformato_Basico,Subformato_Intermedio,Subformato_Plus
0,0,1,369772,2020-08-01,5.0,21.063200,8,1,2020,0,1,1,0,0
1,1,1,369772,2020-08-02,3.0,21.388667,8,2,2020,0,1,1,0,0
2,2,1,369772,2020-08-03,5.0,21.376199,8,3,2020,0,1,1,0,0
3,3,1,369772,2020-08-04,3.0,21.360367,8,4,2020,0,1,1,0,0
4,4,1,369772,2020-08-05,1.0,21.397100,8,5,2020,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852749,87,1,2098857,2021-10-27,8.0,27.003750,10,27,2021,0,1,0,0,1
852750,88,1,2098857,2021-10-28,18.0,26.911667,10,28,2021,0,1,0,0,1
852751,89,1,2098857,2021-10-29,13.0,24.463077,10,29,2021,0,1,0,0,1
852752,90,1,2098857,2021-10-30,17.0,25.404118,10,30,2021,0,1,0,0,1


In [155]:
storeFirstDataframeFeatureDates.describe(include='all')

C:\Users\alan_\AppData\Local\Temp\ipykernel_26496\938346869.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,Unnamed: 0,Tienda,Producto,id_fec_diaria,Venta,Precio,Mes,Day,Year,Formato_Hiper,Formato_Super,Subformato_Basico,Subformato_Intermedio,Subformato_Plus
count,852754.000000,852754.0,8.527540e+05,852754,852754.000000,852754.000000,852754.000000,852754.000000,852754.0,852754.000000,852754.000000,852754.000000,852754.000000,852754.000000
unique,NaN,NaN,NaN,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,2021-08-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,4761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,2020-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2021-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,45.552659,1.0,5.120868e+06,NaN,71.219740,59.971926,9.001217,15.852056,2020.5,0.759856,0.240144,0.399843,0.359532,0.240625
std,26.532938,0.0,4.911186e+06,NaN,334.341760,68.360289,0.820393,8.864996,0.5,0.427171,0.427171,0.489866,0.479863,0.427463
min,0.000000,1.0,1.160000e+02,NaN,0.076000,0.008500,8.000000,1.000000,2020.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,1.0,1.035460e+06,NaN,4.000000,23.260808,8.000000,8.000000,2020.0,1.000000,0.000000,0.000000,0.000000,0.000000


In [156]:
storeFirstDataframeFeatureDates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852754 entries, 0 to 852753
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Unnamed: 0             852754 non-null  int64         
 1   Tienda                 852754 non-null  int64         
 2   Producto               852754 non-null  int64         
 3   id_fec_diaria          852754 non-null  datetime64[ns]
 4   Venta                  852754 non-null  float64       
 5   Precio                 852754 non-null  float64       
 6   Mes                    852754 non-null  int64         
 7   Day                    852754 non-null  int64         
 8   Year                   852754 non-null  int64         
 9   Formato_Hiper          852754 non-null  uint8         
 10  Formato_Super          852754 non-null  uint8         
 11  Subformato_Basico      852754 non-null  uint8         
 12  Subformato_Intermedio  852754 non-null  uint

In [157]:

storeFirstDataframeFeatureDates.rename(columns = {'Unnamed: 0':'Evento'}, inplace = True)

In [158]:
storeFirstDataframeFeatureDates

,Evento,Tienda,Producto,id_fec_diaria,Venta,Precio,Mes,Day,Year,Formato_Hiper,Formato_Super,Subformato_Basico,Subformato_Intermedio,Subformato_Plus
0,0,1,369772,2020-08-01,5.0,21.063200,8,1,2020,0,1,1,0,0
1,1,1,369772,2020-08-02,3.0,21.388667,8,2,2020,0,1,1,0,0
2,2,1,369772,2020-08-03,5.0,21.376199,8,3,2020,0,1,1,0,0
3,3,1,369772,2020-08-04,3.0,21.360367,8,4,2020,0,1,1,0,0
4,4,1,369772,2020-08-05,1.0,21.397100,8,5,2020,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852749,87,1,2098857,2021-10-27,8.0,27.003750,10,27,2021,0,1,0,0,1
852750,88,1,2098857,2021-10-28,18.0,26.911667,10,28,2021,0,1,0,0,1
852751,89,1,2098857,2021-10-29,13.0,24.463077,10,29,2021,0,1,0,0,1
852752,90,1,2098857,2021-10-30,17.0,25.404118,10,30,2021,0,1,0,0,1


In [159]:
storeFirstDataframeFeatureDates['Mes'] = list_months
storeFirstDataframeFeatureDates['Day'] = list_days
storeFirstDataframeFeatureDates['Year'] = list_years
storeFirstDataframeFeatureDates['DayOfWeek'] = storeFirstDataframeFeatureDates['id_fec_diaria'].dt.dayofweek

In [160]:
storeFirstDataframeFeatureDates.head()

,Evento,Tienda,Producto,id_fec_diaria,Venta,Precio,Mes,Day,Year,Formato_Hiper,Formato_Super,Subformato_Basico,Subformato_Intermedio,Subformato_Plus,DayOfWeek
0,0,1,369772,2020-08-01,5.0,21.063200,8,1,2020,0,1,1,0,0,5
1,1,1,369772,2020-08-02,3.0,21.388667,8,2,2020,0,1,1,0,0,6
2,2,1,369772,2020-08-03,5.0,21.376199,8,3,2020,0,1,1,0,0,0
3,3,1,369772,2020-08-04,3.0,21.360367,8,4,2020,0,1,1,0,0,1
4,4,1,369772,2020-08-05,1.0,21.397100,8,5,2020,0,1,1,0,0,2


In [161]:
storeFirstDataframeFeatureDates.tail()

,Evento,Tienda,Producto,id_fec_diaria,Venta,Precio,Mes,Day,Year,Formato_Hiper,Formato_Super,Subformato_Basico,Subformato_Intermedio,Subformato_Plus,DayOfWeek
852749,87,1,2098857,2021-10-27,8.0,27.003750,10,27,2021,0,1,0,0,1,2
852750,88,1,2098857,2021-10-28,18.0,26.911667,10,28,2021,0,1,0,0,1,3
852751,89,1,2098857,2021-10-29,13.0,24.463077,10,29,2021,0,1,0,0,1,4
852752,90,1,2098857,2021-10-30,17.0,25.404118,10,30,2021,0,1,0,0,1,5
852753,91,1,2098857,2021-10-31,23.0,23.815652,10,31,2021,0,1,0,0,1,6


Tenemos un total de 2405 productos distintos. Trabajar con one hot encoding nos resultaría problemático. Si bien es posible trabajar con vectores tan grandes, esto podría traer como consecuencia un sobre entrenamiento del modelo en cuestión, por lo tanto tendremos qué trabajar con la ID de los mismos como valor numérico. Aunque, debido a la dimensión tan grande de las mismas lo ideal sería normalizar dichos valores numéricos

In [162]:
storeFirstDataframeFeatureDates.to_csv('./rawData/data_prueba_Forecasting_Large_Dataframe.csv')